# Imports

In [ ]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

In [ ]:
# Para melhor organização dos dados em tabelas/matrizes
import pandas as pd

# Para visualização dos dados
import seaborn as sns
import matplotlib.pyplot as plt
from kneed import KneeLocator # Número de grupos ideal do agrupamento
from sklearn.cluster import KMeans, DBSCAN # Métodos de Agrupamento
from sklearn_som.som import SOM

# Para analizar as silhuetas
from sklearn.metrics import silhouette_score

# Para calcular
import math

import numpy as np

# Dados

In [ ]:
# O arquivo de dados não está incluso no Git por ser grande demais, então deve ser baixado separadamente
#dados = pd.read_csv('creditcard.csv', sep=',', na_values=["", " ", "NA", "N/A"])
# Dados processados na questão 1
dados = pd.read_csv('creditcard_processado.csv', sep=',', na_values=["", " ", "NA", "N/A"])
dados

In [ ]:
n_instancias = len(dados.index)

# Agrupamentos

In [ ]:
# O correto seria considerar o quadrado da metade de instâncias como o número de clusters,
#  só que isso dá um número absurdo (cerca de 300), então vou reduzir para
#  ficar viável os testes (em quesito de tempo de processamento)
#clusters_max = int(math.sqrt( len(dados.index)/2 ))
clusters_max = 20

In [ ]:
cores = []
cores.append((0.0, 0.0, 0.0)) # Preto para Centroids
cores.append((0.9, 0.1, 0.1)); cores.append((0.1, 0.9, 0.1)); cores.append((0.1, 0.1, 0.9))
cores.append((0.4, 0.4, 0.1)); cores.append((0.1, 0.4, 0.4)); cores.append((0.4, 0.1, 0.4))
cores.append((0.8, 0.4, 0.1)); cores.append((0.1, 0.8, 0.4)); cores.append((0.8, 0.1, 0.4))
cores.append((0.4, 0.8, 0.1)); cores.append((0.1, 0.4, 0.8)); cores.append((0.4, 0.1, 0.8))
cores.append((0.8, 0.3, 0.3)); cores.append((0.3, 0.8, 0.3)); cores.append((0.3, 0.3, 0.8))
cores.append((0.7, 0.5, 0.5)); cores.append((0.5, 0.7, 0.5)); cores.append((0.5, 0.5, 0.7))
cores.append((0.6, 0.1, 0.1)); cores.append((0.1, 0.6, 0.1)); cores.append((0.1, 0.1, 0.6))
cores.append((0.7, 0.7, 0.1)); cores.append((0.1, 0.7, 0.7)); cores.append((0.7, 0.1, 0.7))
cores.append((0.8, 0.6, 0.6)); cores.append((0.6, 0.8, 0.6)); cores.append((0.6, 0.6, 0.8))

## Distribuição Real

In [ ]:
dados_copia = dados.copy()
dados_copia_numpy = dados_copia.to_numpy()
dados_classe = dados['Class']
#print(dados_classe)
#dados_copia = dados_copia.drop(columns=['Class'])
#dados_copia = dados_copia.drop(columns=['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10'])
#dados_copia = dados_copia.drop(columns=['V1', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10'])
#dados_copia = dados_copia.drop(columns=['V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20'])
#dados_copia = dados_copia.drop(columns=['V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28'])

# Matriz de gráficos scatter
#sns.pairplot(dados_copia, hue='Class', height=3.5)

#plt.show()

In [ ]:
# Classes
plt.scatter(dados_copia_numpy[dados_classe == 0, 0], dados_copia_numpy[dados_classe == 0, 1], s = 10, color = cores[2], label = f'Verídico')
plt.scatter(dados_copia_numpy[dados_classe == 1, 0], dados_copia_numpy[dados_classe == 1, 1], s = 10, color = cores[1], label = f'Fraude')
plt.legend()
plt.show()

## K-Means

In [ ]:
dados_kmeans = dados.copy()
dados_kmeans = dados_kmeans.drop(columns=['Class'])

In [ ]:
#print(dados_kmeans)
#print(type(dados_kmeans))
dados_kmeans_numpy = dados_kmeans.to_numpy()
#print(type(dados_kmeans))

### Encontrar número ideal de clusters

In [ ]:
wcss = []
for i in range(2, clusters_max):
  modelo_kmeans = KMeans(n_clusters=i, random_state=69)
  modelo_kmeans.fit(dados_kmeans)

  wcss.append(modelo_kmeans.inertia_)

In [ ]:
kl = KneeLocator(range(2, clusters_max), wcss, curve="convex", direction="decreasing")
clusters_elbow = int(kl.elbow)
clusters_elbow

Visualização com o método elbow

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(2, clusters_max), wcss)
plt.xticks(range(2, clusters_max))
plt.title('Elbow method')
plt.xlabel("Number of Clusters")
plt.show()

### Resultados

Com número de clusters ideal encontrado

In [ ]:
modelo_kmeans_ideal = KMeans(n_clusters=clusters_elbow, random_state=69)
saida_kmeans_ideal = modelo_kmeans_ideal.fit_predict(dados_kmeans_numpy)

In [ ]:
# Classes
for i in range(clusters_elbow):
    plt.scatter(dados_kmeans_numpy[saida_kmeans_ideal == (i), 0], dados_kmeans_numpy[saida_kmeans_ideal == (i), 1], s = 10, color = cores[(i+1)], label = f'Grupo{(i+1)}')
# Centróides
plt.scatter(modelo_kmeans_ideal.cluster_centers_[:, 0], modelo_kmeans_ideal.cluster_centers_[:, 1], s = 10, color = cores[0], label = 'Centroids')
plt.show()

Com número de clusters reais

In [ ]:
modelo_kmeans_real = KMeans(n_clusters=2, random_state=69)
saida_kmeans_real = modelo_kmeans_real.fit_predict(dados_kmeans_numpy)

In [ ]:
# Classes
for i in range(2):
    plt.scatter(dados_kmeans_numpy[saida_kmeans_real == (i), 0], dados_kmeans_numpy[saida_kmeans_real == (i), 1], s = 10, color = cores[(i+1)], label = f'Grupo{(i+1)}')
# Centróides
plt.scatter(modelo_kmeans_real.cluster_centers_[:, 0], modelo_kmeans_real.cluster_centers_[:,1], s = 10, color = cores[0], label = 'Centroids')

plt.show()

## DBSCAN

In [ ]:
dados_dbscan = dados.copy()
dados_dbscan = dados_dbscan.drop(columns=['Class'])
dados_dbscan_numpy = dados_dbscan.to_numpy()
dados_dbscan_numpy = dados_dbscan_numpy.astype(np.float32) # Para reduzir tamanho
#dados_dbscan_teste = dados_dbscan.sample(n=50000, random_state=69).to_numpy()

In [ ]:
modelo_dbscan = DBSCAN(eps=0.125, min_samples=max(2, int(n_instancias/10000)), n_jobs=1)

In [ ]:
saida_dbscan = modelo_dbscan.fit_predict(dados_dbscan_numpy)

In [ ]:
clusters_dbscan = len(set(saida_dbscan))
print(set(saida_dbscan))
print(clusters_dbscan)

In [ ]:
# Barulho encontrado
plt.scatter(dados_dbscan_numpy[saida_dbscan == (-1), 0], dados_dbscan_numpy[saida_dbscan == (-1), 1], s = 10, color = cores[0], label = 'Noise')

# Classes
for i in range(clusters_dbscan-1):
    plt.scatter(dados_dbscan_numpy[saida_dbscan == (i), 0], dados_dbscan_numpy[saida_dbscan == (i), 1], s = 10, color = cores[(i+1)], label = f'Grupo{(i+1)}')

plt.label()
plt.show()

## SOM

In [ ]:
dados_som = dados.copy()
dados_som = dados_som.drop(columns=['Class'])
dados_som_numpy = dados_som.to_numpy()

In [ ]:
modelo_som = SOM(dim= 20)
saida_som = modelo_som.fit_predict(dados_som_numpy)

In [ ]:
#print(set(saida_som))
#print(len( set(saida_som) ))
clusters_som = len(set(saida_som))
clusters_som

In [ ]:
# Classes
for i in range(clusters_som):
    plt.scatter(dados_som_numpy[saida_som == (i), 0], dados_som_numpy[saida_som == (i), 1], s = 10, color = cores[(i+1)], label = f'Grupo{(i+1)}')
# Centróides
#plt.scatter(modelo_som.cluster_centers_[:, 0], modelo_som.cluster_centers_[:,1], s = 10, color = cores[0], label = 'Centroids')

plt.show()

# Comparações e Medidas

## Índice de Silhueta

In [ ]:
score_kmeans_ideal = silhouette_score(dados_kmeans, saida_kmeans_ideal)
score_kmeans_real = silhouette_score(dados_kmeans, saida_kmeans_real)
score_dbscan = silhouette_score(dados_dbscan, saida_dbscan)
score_som = silhouette_score(dados_som, saida_som)

## Sum of Squared Error (SSE)

In [ ]:
# Calculate SSE
sse_kmeans_ideal = np.sum((dados_classe - saida_kmeans_ideal) ** 2)
sse_kmeans_real = np.sum((dados_classe - saida_kmeans_real) ** 2)
sse_dbscan = np.sum((dados_classe - saida_dbscan) ** 2)
sse_som = np.sum((dados_classe - saida_som) ** 2)

## Comparação Final

In [ ]:
comparacoes = pd.DataFrame(index=['KMeans Ideal', 'KMeans Real', 'DBSCAN', 'SOM'], columns=['Clusters', 'Silhouette', 'SSE'])

comparacoes.loc['KMeans Ideal', 'Clusters'] = clusters_elbow; comparacoes.loc['KMeans Ideal', 'Silhouette'] = score_kmeans_ideal; comparacoes.loc['KMeans Ideal', 'SSE'] = sse_kmeans_ideal
comparacoes.loc['KMeans Real', 'Clusters'] = 2;               comparacoes.loc['KMeans Real', 'Silhouette'] = score_kmeans_real;   comparacoes.loc['KMeans Real', 'SSE'] = sse_kmeans_ideal
comparacoes.loc['DBSCAN', 'Clusters'] = clusters_dbscan;      comparacoes.loc['DBSCAN', 'Silhouette'] = score_dbscan;             comparacoes.loc['DBSCAN', 'SSE'] = sse_dbscan
comparacoes.loc['SOM', 'Clusters'] = clusters_som;            comparacoes.loc['SOM', 'Silhouette'] = score_som;                   comparacoes.loc['SOM', 'SSE'] = sse_som

In [ ]:
comparacoes